The particle dynamics in a general velocity field $ \mathbf{u}(\mathbf{x}, t) = \begin{pmatrix} u(\mathbf{x}, t) \\ v(\mathbf{x}, t) \\  w(\mathbf{x}, t) \end{pmatrix} $. This function evaluates the velocity field $ \mathbf{u}(\mathbf{x}, t) $, at point $ \mathbf{x} $ at time $ t $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (3, NxNyNz) | $ \mathbf{x} $ |
| X | array (NY, NX, NZ) | X-meshgrid|
| Y | array (NY, NX, NZ) | Y-meshgrid|
| Z | array (NY, NX, NZ) | Y-meshgrid|
| Interpolant_u | RectangularGridInterpolator | Interpolant object for $ u(\mathbf{x}, t)  $ |
| Interpolant_v | RectangularGridInterpolator | Interpolant object for $ v(\mathbf{x}, t)  $ |
| Interpolant_w | RectangularGridInterpolator | Interpolant object for $ w(\mathbf{x}, t)  $ |
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in z |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| vel | array (3, NxNyNz) | $ \begin{pmatrix} u(\mathbf{x}, t) \\ v(\mathbf{x}, t) \\ w(\mathbf{x}, t) \end{pmatrix} $ |

In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add utils folder to current working path
sys.path.append(parent_directory+"/utils")

In [ ]:
# Import numpy
import numpy as np

In [1]:
def velocity(t, x, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady):
    '''
    Evaluate the interpolated velocity field over the specified spatial locations at the specified time.
    
    Parameters:
        t: array (N,),  time array  
        x: array (3,Npoints),  array of ICs
        X: array (NY, NX, NZ)  X-meshgrid
        Y: array (NY, NX, NZ)  Y-meshgrid 
        Z: array (NY, NX, NZ)  Z-meshgrid
        Interpolant_u: Interpolant object for u(x, t)
        Interpolant_v: Interpolant object for v(x, t)
        Interpolant_w: Interpolant object for w(x, t)
        periodic: list of 3 bools, periodic[i] is True if the flow is periodic in the ith coordinate.
        bool_unsteady:  specifies if velocity field is unsteady/steady
    
    Returns:

        vel = [u, v, w]: concatenated velocityies, same shape as x
    '''
    x_swap = np.zeros((x.shape[1], x.shape[0]))
    x_swap[:,0] = x[1,:]
    x_swap[:,1] = x[0,:]
    x_swap[:,2] = x[2,:] 
    
    # check if periodic in x
    if periodic[0]:
        
        x_swap[:,1] = (x[0,:]-X[0, 0, 0])%(X[0, -1, 0]-X[0, 0, 0])+X[0, 0, 0]

    
    # check if periodic in y
    if periodic[1]:
        
        x_swap[:,0] = (x[1,:]-Y[0, 0, 0])%(Y[-1, 0, 0]-Y[0, 0, 0])+Y[0, 0, 0]
    
    # check if periodic in z
    if periodic[2]:
        
        x_swap[:,2] = (x[2,:]-Z[0, 0, 0])%(Z[0, 0, -1]-Z[0, 0, 0])+Z[0, 0, 0]
        
        
    if bool_unsteady:
    
        u = Interpolant_u(np.append(x_swap, t*np.ones((x_swap.shape[0], 1)), axis = 1))
        v = Interpolant_v(np.append(x_swap, t*np.ones((x_swap.shape[0], 1)), axis = 1))
        w = Interpolant_w(np.append(x_swap, t*np.ones((x_swap.shape[0], 1)), axis = 1))
    
    else:
               
        u = Interpolant_u(x_swap)
        v = Interpolant_v(x_swap)
        w = Interpolant_w(x_swap)
    
    vel = np.array([u, v, w])
    
    return vel